In [11]:
# from datasets import load_dataset
# from collections import Counter

# dataset = load_dataset("glue", "qnli", split="validation")
# print(dataset)

# # Majority class baseline
# counts = Counter(dataset["label"])
# print(counts)
# most_freq_label = counts.most_common(1)[0][0]
# print(most_freq_label)
# print(f"Majority class baseline accuracy: {counts[most_freq_label] / len(dataset) * 100:.2f}")

Found cached dataset glue (/admin/home-augustas/.cache/huggingface/datasets/glue/qnli/1.0.0/dacbe3125aa31d7f70367a07a8a9e72a5a0bfeb5fc42e75c9db75b96da6053ad)


Dataset({
    features: ['question', 'sentence', 'label', 'idx'],
    num_rows: 5463
})
Counter({1: 2761, 0: 2702})
1
Majority class baseline accuracy: 50.54


In [1]:
from transformers import AutoModel, AutoConfig

In [6]:
model_name = "microsoft/deberta-v3-large"
model_name = "microsoft/deberta-v2-xxlarge"
model_name = "EleutherAI/gpt-j-6b"
model_name = "allenai/unifiedqa-v2-t5-11b-1363200"
model_name = "allenai/unifiedqa-v2-t5-3b-1363200"
model_name = "databricks/dolly-v2-3b"
model_name = "gpt2"
model_name = "gpt2-xl"

model_cfg = AutoConfig.from_pretrained(model_name)
model_cfg.is_encoder_decoder, model_cfg.is_decoder

(False, False)

In [7]:
model_cfg.torch_dtype

In [8]:
model_cfg.architectures

['GPT2LMHeadModel']

In [9]:
model_cfg.max_position_embeddings

1024

In [10]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained(model_name)
tokenizer

GPT2TokenizerFast(name_or_path='gpt2-xl', vocab_size=50257, model_max_length=1024, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'bos_token': '<|endoftext|>', 'eos_token': '<|endoftext|>', 'unk_token': '<|endoftext|>'}, clean_up_tokenization_spaces=True)

In [11]:
# model = AutoModel.from_pretrained(model_name)
# print(f"# params: {sum(p.numel() for p in model.parameters()):,d}")

In [12]:
# from transformers import T5ForConditionalGeneration

# model = T5ForConditionalGeneration.from_pretrained(model_name)
# print(f"# params: {sum(p.numel() for p in model.parameters()):,d}")

# params: 2,851,598,336


In [11]:
from transformers import GPTNeoXForCausalLM

model = GPTNeoXForCausalLM.from_pretrained(model_name)
print(f"# params: {sum(p.numel() for p in model.parameters()):,d}")

# params: 2,775,086,080


In [45]:
model.gpt_neox.embed_in

Embedding(50280, 2560)

In [42]:
import torch

model.to("cuda:0")

# More than 2048 does not work
x = torch.randint(0, tokenizer.vocab_size, (2, 2048)).to("cuda:0")
x

with torch.no_grad():
    outputs = model(x)

outputs.keys()

odict_keys(['logits', 'past_key_values'])

In [43]:
outputs.logits.shape

torch.Size([2, 2048, 50280])